# IBM Data Science Certificate Capstone Project

## Boston, Massachusetts Subway Station Analysis

### Introduction

The city of Boston, Massachusetts and its' surrounding areas (ex. Cambridge) is a thriving city on the east coast of the U.S. Many industries, especially healthcare and biopharma have placed their headquarters <br> in the region. The region is also home to dozens of colleges and universities that bring tens of thousands of students to the area. Unfortunately, this economic actvity comes with a cost, congestion. The Boston <br> metro area is notoroius for its traffic. Depending on the study, it has been ranked as the worst traffic in the entire United States. In fact, the Boston Globe published a wide ranging analysis of the problem in <br> November 2019 called, "Seeing Red" that documented the problem and some potential solutions.

One piece of the solution to this problem is for more people to use public transporation such as trains and buses. In the Boston area, one key method of public transit is the MBTA, or Massachusetts Bay Transporation <br> Authority. The MBTA offers subways, ferries, commuter rail, and buses as means of moving about the area. In this project, we'd like to focus specifically  on the subway. We would like to know what types of businesses <br> (and how many) are near each subway station. This analysis will give us an idea of which stations might be best to live and/or work near.

Possible audiences for our analysis include people moving to Boston from other areas, those moving within the city, and perhaps, most importantly, those considering getting rid of their car in favor of commuting on <br> public transit. We want to help these individuals decide which subway stations might be best suited to live near. The analysis could also be useful for businesses in deciding which stations to locate their offices near. <br> A key benefit for many employees is the ability to go the gym after work, grab a drink with colleagues, or have lots of lunch options within walking distance.

### Data

To complete this analysis, we will use geographic data for each subway station as well as the FourSquare API to explore venue information within a specified radius of each station. We will identify the latitude <br> and longitude of each MBTA subway station in the region. Then we will use the FourSquare API to pull the types and number of venues within a specified radius of each station. Next, we will clean and organize <br> the data using pandas to provide a useful subset of the types of businesses that individuals would find useful about an area. We will also use folium to map the stations as a convenient visualization for our <br> audience. This will give users a birds eye view of all stations and where they are in relation to one another. Lastly, we will group and rank each station by the number and types of venues. 

If possible, we will also attempt to cluster the stations based on those that have the most venues within close proximity. This will provide for a better idea of the stations that are most similar and give users a <br> series of possible options to choose from. 